In [1]:
import torch
import torchvision.transforms as tvt
from torch.optim import Adam

from torch.utils.data import DataLoader, Subset
from torchvision.datasets import CIFAR10
from tqdm.notebook import tqdm
from torchmetrics import Accuracy, AUROC
from torch.optim.lr_scheduler import CosineAnnealingLR

from pytorch_ood.utils import is_known, is_unknown, contains_known, contains_unknown
from pytorch_ood.loss import ObjectosphereLoss
from pytorch_ood.dataset.img import Textures, CIFAR10C, LSUNCrop, LSUNResize, TinyImageNetResize, TinyImageNetCrop
from pytorch_ood.dataset.img import TinyImages300k
from pytorch_ood.model import WideResNet
from pytorch_ood.transforms import ToRGB, ToUnknown
from pytorch_ood.metrics import OODMetrics

In [2]:
torch.manual_seed(123)

mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

trans = tvt.Compose([ToRGB(), tvt.Resize((32,32)), tvt.ToTensor(), tvt.Normalize(mean, std)])

# setup data
dataset_in_train = CIFAR10(root="data", train=True, download=True, transform=trans)
# dataset_in_test = CIFAR10(root="data", train=False, transform=trans)
dataset_out_train = Subset(
    TinyImages300k(root="/home/ki/datasets/", transform=trans, target_transform=ToUnknown(), download=True), indices=range(120000))

train_loader = DataLoader(dataset_in_train + dataset_out_train, batch_size=128, shuffle=True)


Files already downloaded and verified


In [3]:
model = WideResNet.from_pretrained("cifar10-pt", num_classes=10)
opti = Adam(params=model.parameters(), lr=0.001)
crit = ObjectosphereLoss()
scheduler = CosineAnnealingLR(opti, T_max=len(train_loader) * 10)
model = model.cuda()

In [4]:

acc = Accuracy(num_classes=10).cuda()
auroc = AUROC(num_classes=2).cuda()

for epoch in range(10):
    bar = tqdm(train_loader)
    for batch in bar:
        x, y = batch
        x, y = x.cuda(), y.cuda()
        z = model(x)

        loss = crit(z, y)
        opti.zero_grad()
        loss.backward()
        opti.step()
        scheduler.step()

        acc.update(z[is_known(y)],y[is_known(y)])
        auroc.update(ObjectosphereLoss.score(z),is_unknown(y))
        bar.set_postfix({"loss": loss.item(), "acc": acc.compute().item(), "auroc": auroc.compute().item()})

    acc.reset()
    auroc.reset()

torch.save(model, "/home/ki/cifar10-objectosphere.pth")

/home/ki/.local/share/anaconda3/envs/myenv2/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


  0%|          | 0/1329 [00:00<?, ?it/s]

/home/ki/.local/share/anaconda3/envs/myenv2/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: `pos_label` automatically set 1.
  warnings.warn(*args, **kwargs)


  0%|          | 0/1329 [00:00<?, ?it/s]

  0%|          | 0/1329 [00:00<?, ?it/s]

...

  0%|          | 0/1329 [00:00<?, ?it/s]

  0%|          | 0/1329 [00:00<?, ?it/s]

  0%|          | 0/1329 [00:00<?, ?it/s]

  0%|          | 0/1329 [00:00<?, ?it/s]

  0%|          | 0/1329 [00:00<?, ?it/s]

In [6]:
dataset_in_test = CIFAR10(root="data", train=False, transform=trans)
dataset_out_test1 = Textures(root="data", download=True, transform=trans)
dataset_out_test2 = LSUNCrop(root="data", download=True, transform=trans)
dataset_out_test3 = LSUNResize(root="data", download=True, transform=trans)
dataset_out_test4 = TinyImageNetResize(root="data", download=True, transform=trans)
dataset_out_test5 = TinyImageNetCrop(root="data", download=True, transform=trans)
dataset_test = dataset_in_test + dataset_out_test1 + dataset_out_test2 + dataset_out_test3 + dataset_out_test4 + dataset_out_test5
test_loader = DataLoader(dataset_test, batch_size=128)

acc = Accuracy(num_classes=10).cuda()
metrics = OODMetrics()

with torch.no_grad():
    for batch in tqdm(test_loader):
        x, y = batch
        x, y = x.cuda(), y.cuda()
        z = model(x)
        acc.update(z[is_known(y)],y[is_known(y)])
        metrics.update(ObjectosphereLoss.score(z),y)

print(acc.compute())
print(metrics.compute())

/home/ki/.local/share/anaconda3/envs/myenv2/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/ki/.local/share/anaconda3/envs/myenv2/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


  0%|          | 0/435 [00:00<?, ?it/s]

tensor(0.8728, device='cuda:0')


/home/ki/.local/share/anaconda3/envs/myenv2/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: `pos_label` automatically set 1.
  warnings.warn(*args, **kwargs)
/home/ki/.local/share/anaconda3/envs/myenv2/lib/python3.8/site-packages/torchmetrics/functional/classification/precision_recall_curve.py:156: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  thresholds = tensor(reversed(thresholds[sl]))


{'AUROC': 0.9616478681564331, 'AUPR-IN': 0.9888243675231934, 'AUPR-OUT': 0.875281035900116, 'ACC95TPR': 0.9318296313285828, 'FPR95TPR': 0.15109999477863312}


In [8]:
from pytorch_ood.dataset.img import Textures, CIFAR10C, LSUNCrop, LSUNResize, TinyImageNetResize, TinyImageNetCrop
cifarc = CIFAR10C(root="data/", subset="all", download=True, transform=trans)
test_loader = DataLoader(cifarc, batch_size=128)

acc = Accuracy(num_classes=10).cuda()

with torch.no_grad():
    for batch in tqdm(test_loader):
        x, y = batch
        x, y = x.cuda(), y.cuda()
        z = model(x)
        acc.update(z[is_known(y)],y[is_known(y)])

print(acc.compute())

  0%|          | 0/1 [00:00<?, ?it/s]

tensor(0.8500, device='cuda:0')
